## 0) Setup & config

In [ ]:
# --- CONFIG: set your input/output paths here ---
# If your CSV is in the same folder as the notebook, just use the filename.
INPUT_PATH  = r'D:\Group_assignment_DL\mansion-energy-forecast\data_raw\dataset_mansion.csv'   # <-- change if needed
OUTPUT_PATH = r'D:\Group_assignment_DL\mansion-energy-forecast\output\dataset_mansion_cleaned.csv'  # <-- change if needed

# Optional: how aggressive to cap outliers on target
LOW_Q  = 0.01   # 1st percentile
HIGH_Q = 0.99   # 99th percentile

# Interpolation limit (hours) for small gaps
INTERP_LIMIT_HOURS = 6


## 1) Imports

In [13]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## 2) Helpers (auto-detect columns)

In [14]:
def detect_time_col(df):
    for name in ['MTime','time','Time','timestamp','Timestamp','Date','date','DATE']:
        if name in df.columns:
            return name
    # fallback: column that parses mostly to datetimes
    for col in df.columns:
        parsed = pd.to_datetime(df[col], errors='coerce', infer_datetime_format=True)
        if parsed.notna().sum() > 0.5 * len(df):
            return col
    return None

def detect_consumption_col(df):
    # common names
    for name in ['Consumption','consumption','CONS','Power','power','Load','load','kWh','kwh']:
        if name in df.columns:
            return name
    # heuristic: name contains key words
    for col in df.columns:
        low = col.lower()
        if any(k in low for k in ['consump','power','load','kwh']):
            return col
    # fallback: pick a numeric column with highest variance
    num = df.select_dtypes(include=[np.number]).columns
    if len(num) > 0:
        return df[num].var().sort_values(ascending=False).index[0]
    return None

def detect_weather_cols(df):
    cols = []
    for col in df.columns:
        low = col.lower()
        if any(k in low for k in ['temp','temperature','humid','humidity','precip','rain','pressure','wind']):
            cols.append(col)
    return cols


## 3) Load (preview → detect columns → load full)

In [15]:
if not os.path.exists(INPUT_PATH):
    raise FileNotFoundError(f"Input not found: {INPUT_PATH}")

# quick preview to detect columns
preview = pd.read_csv(INPUT_PATH, nrows=500)
time_col = detect_time_col(preview)
if time_col is None:
    raise ValueError("Could not detect a datetime column. Please check your CSV.")

# load full with time parsed
df_raw = pd.read_csv(INPUT_PATH)
df_raw[time_col] = pd.to_datetime(df_raw[time_col], errors='coerce', infer_datetime_format=True)
df_raw = df_raw.loc[df_raw[time_col].notna()].copy()
df_raw = df_raw.set_index(time_col).sort_index()

print("Detected time column:", time_col)
print(df_raw.head(3))
print(df_raw.tail(3))


Detected time column: MTime
                     Unnamed: 0 Consumption  Latitude  Longitude  \
MTime                                                              
2015-12-31 22:00:00           0        3,21  62.39758   25.67087   
2015-12-31 23:00:00           1        2,88  62.39758   25.67087   
2016-01-01 00:00:00           2         3,7  62.39758   25.67087   

                                    Place  AirTemperature(degC)  \
MTime                                                             
2015-12-31 22:00:00  Jyväskylä lentoasema                  -4.3   
2015-12-31 23:00:00  Jyväskylä lentoasema                  -4.6   
2016-01-01 00:00:00  Jyväskylä lentoasema                  -4.5   

                     HighestTemperature(degC)  LowestTemperature(degC)  \
MTime                                                                    
2015-12-31 22:00:00                      -4.2                     -4.5   
2015-12-31 23:00:00                      -4.4                     -4.6   

C:\Users\Provati\AppData\Local\Temp\ipykernel_4688\3889547083.py:12: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_raw[time_col] = pd.to_datetime(df_raw[time_col], errors='coerce', infer_datetime_format=True)


## 4) Basic checks & resample to hourly

In [16]:
# drop duplicate timestamps (keep first)
if df_raw.index.duplicated().sum() > 0:
    df_raw = df_raw[~df_raw.index.duplicated(keep='first')]

# resample to continuous hourly index
df = df_raw.resample('H').asfreq()

# detect target & weather columns
consum_col   = detect_consumption_col(df)
weather_cols = detect_weather_cols(df)

print("Detected consumption column:", consum_col)
print("Detected weather columns:", weather_cols if weather_cols else "None")

# to numeric (safe)
df[consum_col] = pd.to_numeric(df[consum_col], errors='coerce')
for c in weather_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# quick NA report
na_report = df.isna().sum().sort_values(ascending=False)
display(na_report.head(10))


Detected consumption column: Consumption
Detected weather columns: ['AirTemperature(degC)', 'HighestTemperature(degC)', 'LowestTemperature(degC)', 'RelativeHumidity(%)', 'WindSpeed(m/s)', 'MaximumWindSpeed(m/s)', 'MinimumWindSpeed(m/s)', 'WindDirection(deg)', 'PrecipitationAmount(mm)', 'MaximumPrecipitationIntensity(mm/h)', 'AirPressure(hPa)']


C:\Users\Provati\AppData\Local\Temp\ipykernel_4688\1857638264.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df_raw.resample('H').asfreq()


Consumption                            55782
PresentWeather(rank)                   32441
MaximumWindSpeed(m/s)                    741
MinimumWindSpeed(m/s)                    741
WindSpeed(m/s)                           618
WindDirection(deg)                       501
MaximumPrecipitationIntensity(mm/h)      321
PrecipitationAmount(mm)                  303
RelativeHumidity(%)                      289
AirPressure(hPa)                         210
dtype: int64

## 5) Create weather_missing flag (based on original data presence)

In [17]:
# Default: assume missing before 2017 (common for your dataset)
df['weather_missing'] = (df.index < pd.Timestamp('2017-01-01')).astype(int)

# If at least one weather column exists in the ORIGINAL (unresampled) data,
# mark missing where that column was NaN originally.
if weather_cols:
    first_w = weather_cols[0]
    raw = pd.read_csv(INPUT_PATH)
    raw[time_col] = pd.to_datetime(raw[time_col], errors='coerce', infer_datetime_format=True)
    raw = raw.set_index(time_col).sort_index().resample('H').asfreq()
    if first_w in raw.columns:
        df['weather_missing'] = raw[first_w].isna().astype(int)

df['weather_missing'].value_counts(dropna=False)


C:\Users\Provati\AppData\Local\Temp\ipykernel_4688\3727162148.py:9: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  raw[time_col] = pd.to_datetime(raw[time_col], errors='coerce', infer_datetime_format=True)
C:\Users\Provati\AppData\Local\Temp\ipykernel_4688\3727162148.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  raw = raw.set_index(time_col).sort_index().resample('H').asfreq()


weather_missing
0    56023
1      208
Name: count, dtype: int64

## 6) Impute consumption & weather (small gaps interpolate → ffill/bfill)

In [18]:
# Interpolate small gaps by time (limit = INTERP_LIMIT_HOURS), then ffill/bfill remaining
df[consum_col] = df[consum_col].interpolate(method='time', limit=INTERP_LIMIT_HOURS)
df[consum_col] = df[consum_col].fillna(method='ffill').fillna(method='bfill')

for c in weather_cols:
    df[c] = df[c].interpolate(method='time', limit=INTERP_LIMIT_HOURS)
    df[c] = df[c].fillna(method='ffill').fillna(method='bfill')

# sanity: no NaNs left in target
assert df[consum_col].isna().sum() == 0, "Target still contains NaNs after imputation."


C:\Users\Provati\AppData\Local\Temp\ipykernel_4688\3547954587.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[consum_col] = df[consum_col].fillna(method='ffill').fillna(method='bfill')
C:\Users\Provati\AppData\Local\Temp\ipykernel_4688\3547954587.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[c] = df[c].fillna(method='ffill').fillna(method='bfill')
C:\Users\Provati\AppData\Local\Temp\ipykernel_4688\3547954587.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[c] = df[c].fillna(method='ffill').fillna(method='bfill')
C:\Users\Provati\AppData\Local\Temp\ipykernel_4688\3547954587.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() i

## 7) Outlier capping (winsorize 1% / 99%)

In [19]:
low = df[consum_col].quantile(LOW_Q)
high = df[consum_col].quantile(HIGH_Q)
df[consum_col] = df[consum_col].clip(lower=low, upper=high)

print(f"Outlier caps applied on {consum_col}: [{LOW_Q:.2%},{HIGH_Q:.2%}] -> [{low:.3f},{high:.3f}]")


Outlier caps applied on Consumption: [1.00%,99.00%] -> [1.000,5.888]


## 8) Add simple, model-agnostic time features (useful for everyone)

In [20]:
df['hour']          = df.index.hour
df['dow']           = df.index.dayofweek
df['month']         = df.index.month
df['is_weekend']    = df['dow'].isin([5,6]).astype(int)
df['heating_season']= df['month'].isin([10,11,12,1,2,3,4]).astype(int)

# optional cyclic encodings (comment out if not needed by all)
df['hour_sin'] = np.sin(2*np.pi*df['hour']/24)
df['hour_cos'] = np.cos(2*np.pi*df['hour']/24)
df['doy_sin']  = np.sin(2*np.pi*df.index.dayofyear/365.25)
df['doy_cos']  = np.cos(2*np.pi*df.index.dayofyear/365.25)


## 9) Save cleaned dataset

In [21]:
# Keep full 2016–2022 (do NOT trim), as agreed for group-wide use
df.to_csv(OUTPUT_PATH, index=True)
print("Saved cleaned dataset to:", OUTPUT_PATH)


Saved cleaned dataset to: D:\Group_assignment_DL\mansion-energy-forecast\data_raw\dataset_mansion_cleaned.csv


## 10) Final summary preview

In [ ]:
print("Time range:", df.index.min(), "→", df.index.max())
print("Rows (hours):", len(df))
print("Target:", consum_col)
print("Weather columns:", weather_cols if weather_cols else "None")

display(df[[consum_col] + (weather_cols[:3] if weather_cols else []) + ['weather_missing','hour','dow','month']].head(10))
display(df[[consum_col] + (weather_cols[:3] if weather_cols else []) + ['weather_missing','hour','dow','month']].tail(10))


## How  teammates can use it

* Everyone loads the same dataset_mansion_cleaned.csv.

* if a model depends on weather, they can filter df[df.weather_missing==0] or keep all and let the model learn the flag.

* use the assignment splits:

      train: 2016-01-01 → 2020-09-30

      test: 2020-10-01 → 2021-09-30

      eval A: 2021-10-01 → 2021-12-31

      eval B: 2022-01-01 → 2022-02-28

      eval C: 2022-03-01 → 2022-05-31